## 검색어 생성

In [8]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    query: Annotated[
        list[str],
        Field(
            ..., 
            max_length=5, 
            min_length=3,
            description="가장 적절한 검색어들의 리스트, 길이 최소 3개/최대 5개", 
        )
    ]
    

In [9]:
# 프롬프트
from langchain_core.prompts import PromptTemplate

query_template = '''
주어진 제목과 설명을 바탕으로, 의미적으로 가장 관련성이 높은 논문과 데이터셋을 찾기 위한 검색어를 생성하세요.

[검색 엔진 제약]
검색은 정확 일치 기반입니다. 쿼리는 짧고 응집력 있게 만드세요.
각 쿼리는 2~3단어, 고유명사나 기술 토픽의 조합을 선호합니다.

[생성 절차]
1. 주제 핵심어 파악: 연구 주제의 주요 객체, 방법론, 도메인, 응용 맥락을 한 문장으로 요약
2. 후보 생성: 2~3단어 쿼리 후보를 8~10개 잠정 생성
3. 필터링 규칙 적용:
  - 일반어, 지나치게 포괄적이거나 모호한 표현 제거(ex: “AI model”, “data analysis”)
  - 12개는 방법론 중심, 12개는 도메인 중심, 1~2개는 응용 시나리오 중심으로 균형 있게 남김
  - 약어만 있는 경우는 배제하되, 널리 쓰이는 고유 약어는 유지(ex: “BERT” 가능, “ML” 단독 불가)
  - 하이픈, 특수문자, 따옴표는 사용하지 않음
4. 최종 선택: 상위 3~5개만 남김

[금지]
- 1단어 쿼리 금지
- 4단어 이상 금지
- 불용어만 남는 조합 금지(ex: “for research”)
- JSON 스키마 위반 금지

[Input]
- 연구 주제: {title}
- 연구 설명: {description}
- 키워드: {keyword}

[Output]
다음 형식의 JSON을 출력하세요:
{{
  "query": [],
}}
'''

query_prompt = PromptTemplate.from_template(query_template)

In [10]:
from langchain_openai import ChatOpenAI
import json

with open("../data/input_data.json", "r", encoding="utf-8") as f:
    input_data = json.load(f)

title, description, keyword = input_data['dataset_title_etc_main'], input_data['dataset_expl_etc_main'], input_data['dataset_kywd_etc_main']

prompt = query_prompt.invoke(
    {
        'title': title, 
        'description': description,
        'keyword': keyword
    }
)

# sLLM
sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(QueryResult)
res = structured_sllm.invoke(prompt)

print(res.query)

['Antarctic gravity core', 'Ross Sea sediments', 'climate change Antarctica', 'marine sedimentation', 'gravity core analysis']
